In [1]:
import os
import sys
import json
import random
import re
import string
import tqdm
import argparse
import numpy as np
import pandas as pd
from multiprocessing import Pool
from rouge_score import rouge_scorer

import transformers
from transformers import AutoTokenizer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer

import torch
from peft import PeftModel

random.seed(42)

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cuda")
tokenizer = GPT2Tokenizer.from_pretrained("stanford-crfm/BioMedLM")
model = GPT2LMHeadModel.from_pretrained("stanford-crfm/BioMedLM").to(device)

In [3]:
args = {
    "batch_dir": "data/gpt3_generations/",
    "seed_tasks_path": "data/help-galaxy-qa.jsonl",
    "num_instructions_to_generate": 1,
    "num_prompt_instructions": 1,
    "request_batch_size": 20,
    "use_clf_seed_tasks_only": False
}

print(args)

def encode_prompt(prompt_instructions, classification=False):
    """Encode multiple prompt instructions into a single string."""
    if classification:
        prompt = "Come up with a series of classification tasks. Try to specify the possible output labels when possible.\n"
    else:
        #prompt = "Come up with a series of tasks:\n"
        prompt = "Come up with a series of tasks: \n"
        #"Come up with a new question using scientific keywords mentioned in existing tasks: \n"
        #"Come up with a series of new tasks using scientific keywords mentioned in existing tasks: \n" 
        #"Come up with a series of new tasks: \n"
    for idx, instruction in enumerate(prompt_instructions):
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        prompt += f"{idx+1}. {instruction}\n"
    return prompt


def sample_machine_instructions(machine_instructions, similarities, n):
    """Sample n machine instructions from a list of machine instructions."""
    return random.sample(machine_instructions, min(n, len(machine_instructions)))


def find_word_in_string(w, s):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search(s)

{'batch_dir': 'data/gpt3_generations/', 'seed_tasks_path': 'data/help-galaxy-qa.jsonl', 'num_instructions_to_generate': 1, 'num_prompt_instructions': 1, 'request_batch_size': 20, 'use_clf_seed_tasks_only': False}


In [4]:
def evaluate(
    instruction,
    input=None,
    temperature=0.1, #0.7, #0.1,
    top_p=0.75, #0.5, #0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128, #128,
    #max_tokens=1024,
    stream_output=False
):
    prompt = instruction
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
    )

    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output #prompter.get_response(output)

In [5]:
def evaluate_pubmedgpt(instruction):
    input_ids = tokenizer.encode(
        instruction, return_tensors="pt").to(device)
    sample_output = model.generate(input_ids, do_sample=True, max_length=128, temperature=0.7, top_k=50)
    print("Output:\n" + 100 * "-")
    return tokenizer.decode(sample_output[0])

In [6]:
seed_tasks = [json.loads(l) for l in open(args["seed_tasks_path"], "r")]
if args["use_clf_seed_tasks_only"]:
    seed_tasks = [t for t in seed_tasks if t["is_classification"]]
seed_instructions = [t["instruction"] for t in seed_tasks]
print(f"Loaded {len(seed_instructions)} human-written seed instructions")

os.makedirs(args["batch_dir"], exist_ok=True)

Loaded 5 human-written seed instructions


In [7]:
machine_instructions = []
batch_inputs = []
for _ in range(args["request_batch_size"]):
    prompt_instructions = []
    prompt_instructions += random.sample(seed_instructions, args["num_prompt_instructions"] - len(prompt_instructions))
    random.shuffle(prompt_instructions)
    prompt = encode_prompt(prompt_instructions, classification=args["use_clf_seed_tasks_only"])
    print(evaluate_pubmedgpt(prompt))
    print("---------")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I setup a bed file (see below), but it does not work. I wonder how to correct it? Thanks. HPV16 1024 1043 HPV_1_LEFT 1 + GCDCARGGDCAYAAYAATGG HPV16 1183 1207 HPV_1_RIGHT 1 - GAAAAATAAACTGTAAATCATATTC

2. I'm going to show you a file of the HPV16 reference genome (NC_001526.2) and a file of the HPV
---------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I setup a bed file (see below), but it does not work. I wonder how to correct it? Thanks. HPV16 1024 1043 HPV_1_LEFT 1 + GCDCARGGDCAYAAYAATGG HPV16 1183 1207 HPV_1_RIGHT 1 - GAAAAATAAACTGTAAATCATATTC

2. To analyze the next sequence you will need a list of features that contain the information you want to extract. This list can be found on the HPV webs
---------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.


...

Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I am trying to run the second step of quantification (i.e. quantifier) following the mapper module of mirDeep2. In the mapping step everything is fine and I get the collapsed reads to be given to the quantifier module. Then, when trying to run the quantifier module, providing mature, precursor and star sequences, the run ends with an error which apparently lacks any suggestions on where the error is. Any clues on what’s going on here? I see a lot of unmapped sequences but this is somewhat expected from a biological point of view as the miRNA fraction is not usually the most abundant in the tissue I am investigating (zebrafish gonads)


---------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I am trying to run the second step of quantification (i.e. quantifier) following the mapper module of mirDeep2. In the mapping step everything is fine and I get the collapsed reads to be given to the quantifier module. Then, when trying to run the quantifier module, providing mature, precursor and star sequences, the run ends with an error which apparently lacks any suggestions on where the error is. Any clues on what’s going on here? I see a lot of unmapped sequences but this is somewhat expected from a biological point of view as the miRNA fraction is not usually the most abundant in the tissue I am investigating (zebrafish gonads)


---------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Input length of input_ids is 179, but `max_length` is set to 128. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I setup a bed file (see below), but it does not work. I wonder how to correct it? Thanks. HPV16 1024 1043 HPV_1_LEFT 1 + GCDCARGGDCAYAAYAATGG HPV16 1183 1207 HPV_1_RIGHT 1 - GAAAAATAAACTGTAAATCATATTC

2. I also want to change the format of the sequence and the header of the file. Thanks. HPV16 1024 1043 HPV_
---------
Output:
----------------------------------------------------------------------------------------------------
Come up with a series of tasks: 
1. I am trying to run the second step of quantification (i.e. quantifier) following the mapper module of mirDeep2. In the mapping step everything is fine and I get the collapsed reads to be given to the quantifier module. Then, when trying to run the quantifier module, providing mature, precursor and star sequences, the run ends with an error which apparently lacks any suggestions on where 